In [70]:
import pandas as pd
import numpy as np

alternatif = {
    "A1": "Emina",
    "A2": "Wardah",
    "A3": "Avoskin",
    "A4": "Whitelab",
    "A5": "Skintific",
    "A6": "Somethinc",
    "A7": "Garnier",
    "A8": "Npure",
    "A9": "Ms glow",
    "A10": "The originote",
}

kriteria = {
    "C1": "Keluhan",
    "C2": "Tipe jenis kulit",
    "C3": "Merk",
    "C4": "Harga",
    "C5": "Usia",
    "C6": "Kemasan",
    "C7": "Kualitas",
}

keluhan = {
    "Wajah Kusam": 1,
    "Wajah Beruntusan": 2,
    "Wajah Jerawat": 3,
    "Wajah Kemerahan": 4,
    "Flek Wajah": 5,
}

jenis_kulit = {
    "Kulit Normal": 1,
    "Kulit Kering": 2,
    "Kulit Berminyak": 3,
    "Kulit Berjerawat": 4,
    "Kulit Kombinasi": 5,
}

nilai_harga = {
    "1": [20000, 50000],
    "2": [50000, 150000],
    "3": [150000, 200000],
    "4": [200000, 400000],
}

nilai_merk = {
    "Sangat Terkenal": 5,
    "Terkenal": 4,
    "Cukup Terkenal": 3,
    "Sedikit Terkenal": 2,
    "Kurang Terkenal": 1,
}

usia = {
    "1": [17, 22],
    "2": [23, 28],
    "3": [29, 35],
    "4": [36, 40],
    "5": [41, 45],
}

kemasan = {
    "Sangat Banyak": 5,
    "Banyak": 4,
    "Cukup": 3,
    "Sedikit": 2,
    "Kurang": 1
}

kualitas = {
    "Sangat Bagus": 5,
    "Bagus": 4,
    "Cukup": 3,
    "Buruk": 2,
    "Sangat Buruk": 1
}

In [76]:
import random as rn
# Membuat Data acak menggunakan fungsi random randint
def generate_data():
    new_data = {"Skincare": list(alternatif.values())}
    
    for key in kriteria.values():
        new_data[key] = []

    for i in range(10):
        new_data["Keluhan"].append(rn.randint(1,5))
        new_data["Tipe jenis kulit"].append(rn.randint(1,5))
        new_data["Merk"].append(rn.randint(1,5))
        new_data["Harga"].append(rn.randint(1,4))  # Hanya 4 rentang harga yang tersedia
        new_data["Usia"].append(rn.randint(1,5))
        new_data["Kemasan"].append(rn.randint(1,5))
        new_data["Kualitas"].append(rn.randint(1,5))
    
    return pd.DataFrame(new_data)

# Normalisasi menggunakan metode SAW
def normalize_saw(df, weights):
    df_saw = df.copy()
    for key, val in weights.items():
        if val["atribut"] == "benefit":
            df_saw[key] = (df_saw[key] / df_saw[key].max()) * val["bobot"]
        elif val["atribut"] == "cost":
            df_saw[key] = (df_saw[key].min() / df_saw[key]) * val["bobot"]
    return df_saw

# Metode TOPSIS
def topsis(df, weights):
    X = df.drop('Skincare', axis=1).values
    norm_X = X / np.linalg.norm(X, axis=0)
    
    A_pos = np.max(norm_X, axis=0)
    A_neg = np.min(norm_X, axis=0)
    
    S_pos = np.sqrt(np.sum((norm_X - A_pos) ** 2, axis=1))
    S_neg = np.sqrt(np.sum((norm_X - A_neg) ** 2, axis=1))
    
    C = S_neg / (S_pos + S_neg)
    return C

# Kombinasi SAW dan TOPSIS
def hybrid_saw_topsis(df_saw, weights, alpha=0.5, beta=0.5):
    df_saw['Total_SAW'] = df_saw.drop("Skincare", axis=1).sum(axis=1)
    C_topsis = topsis(df_saw, weights)
    
    df_saw['Closeness_TOPSIS'] = C_topsis
    df_saw['Final_Score'] = alpha * df_saw['Total_SAW'] + beta * df_saw['Closeness_TOPSIS']
    
    return df_saw.sort_values(by='Final_Score', ascending=False)

In [ ]:
# Main
new_df = generate_data()
new_df

In [80]:
weights_saw = {
    "Keluhan": {"bobot": 0.20, "atribut": "benefit"},
    "Tipe jenis kulit": {"bobot": 0.20, "atribut": "benefit"},
    "Harga": {"bobot": 0.10, "atribut": "cost"},
    "Merk": {"bobot": 0.15, "atribut": "benefit"},
    "Usia": {"bobot": 0.15, "atribut": "benefit"},
    "Kemasan": {"bobot": 0.10, "atribut": "benefit"},
    "Kualitas": {"bobot": 0.10, "atribut": "benefit"}
}
df_saw = normalize_saw(new_df, weights_saw)
df_hybrid = hybrid_saw_topsis(df_saw, weights_saw)
df_hybrid[["Skincare", "Final_Score"]]

,Skincare,Final_Score
3,Whitelab,0.779818
9,The originote,0.624760
0,Emina,0.563442
5,Somethinc,0.532699
4,Skintific,0.502706
1,Wardah,0.426538
7,Npure,0.414384
2,Avoskin,0.401007
6,Garnier,0.400155
8,Ms glow,0.314065


In [139]:
# Mencari skincare terbaik untuk user
def find_best_skincare(user_input, df, weights, asc=True):
    df_user = pd.DataFrame(user_input, index=[0])
    df_saw = normalize_saw(df, weights)
    df_hybrid = hybrid_saw_topsis(df_saw, weights)
    df_user_saw = normalize_saw(df_user, weights)
    
    df_hybrid['User_Distance'] = np.linalg.norm(df_hybrid[kriteria.values()].values - df_user_saw[kriteria.values()].values, axis=1)
    
    df_hybrid = df_hybrid.sort_values(by='User_Distance', ascending=asc)
    
    return df_hybrid[["Skincare", "User_Distance"]]

In [125]:
# user input
user_input = {}
for key, criterion in kriteria.items():
    if criterion == "Harga":
        user_input[criterion] = rn.randint(1,4)
    else:
        user_input[criterion] = rn.randint(1,5)
user_input

{'Keluhan': 3,
 'Tipe jenis kulit': 3,
 'Merk': 4,
 'Harga': 2,
 'Usia': 4,
 'Kemasan': 2,
 'Kualitas': 5}

In [140]:
best_skincare = find_best_skincare(user_input, new_df, weights_saw)

print("Skincare yang paling sesuai untuk user:")
best_skincare#.reset_index(drop=True)

Skincare yang paling sesuai untuk user:


,Skincare,User_Distance
3,Whitelab,0.100000
9,The originote,0.122168
5,Somethinc,0.181230
0,Emina,0.192988
2,Avoskin,0.197597
4,Skintific,0.217715
7,Npure,0.229226
8,Ms glow,0.247678
6,Garnier,0.257294
1,Wardah,0.257342
